In [2]:
import tensorflow as tf
import seaborn as sns
import numpy as np

from PIL import Image
import glob
from collections import defaultdict
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
IMG_SIZE = (94, 125)
def pixels_from_path(file_path):
    im = Image.open(file_path)
    im = im.resize(IMG_SIZE)
    np_im = np.array(im)
    return np_im

D:\Nauka\Magisterskie\!sem3\MSI\projekt\dataset

In [4]:
glob.glob('D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset/*')

['D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 000.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 001.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 002.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 003.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 004.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 005.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 006.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 007.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 008.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 009.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 010.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 011.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 012.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 013.PNG',
 'D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset\\left 014.PNG',
 'D:/Nauka

In [5]:
shape_counts = defaultdict(int)
for i, cat in enumerate(glob.glob('D:/Nauka/Magisterskie/!sem3/MSI/projekt/lewo/*')[:1000]):
    if i%100==0:
        print(i)
    img_shape = pixels_from_path(cat).shape
    shape_counts[str(img_shape)]= shape_counts[str(img_shape)]+ 1

0
100
200
300
400
500
600
700
800
900


In [6]:
shape_items = list(shape_counts.items())
shape_items.sort(key = lambda x: x[1])
shape_items.reverse()

In [7]:
validation_size = 0.1
img_size = IMG_SIZE 
num_channels = 3 
sample_size = 2422 

In [8]:
len(glob.glob('D:/Nauka/Magisterskie/!sem3/MSI/projekt/dataset/*'))

2422

In [9]:
pixels_from_path(glob.glob('D:/Nauka/Magisterskie/!sem3/MSI/projekt/lewo/*')[5]).shape

(125, 94, 4)

In [10]:
SAMPLE_SIZE = 1000
left_train_set = np.asarray([pixels_from_path(left) for left in glob.glob('D:/Nauka/Magisterskie/!sem3/MSI/projekt/lewo/*')[:SAMPLE_SIZE]])
right_train_set = np.asarray([pixels_from_path(right) for right in glob.glob('D:/Nauka/Magisterskie/!sem3/MSI/projekt/prawo/*')[:SAMPLE_SIZE]])

In [11]:
VALID_SIZE = 200
left_valid_set = np.asarray([pixels_from_path(left) for left in glob.glob('D:/Nauka/Magisterskie/!sem3/MSI/projekt/lewo/*')[-VALID_SIZE:]])
right_valid_set = np.asarray([pixels_from_path(right) for right in glob.glob('D:/Nauka/Magisterskie/!sem3/MSI/projekt/prawo/*')[-VALID_SIZE:]])

In [12]:
x_train = np.concatenate([left_train_set, right_train_set])
labels_train = np.asarray([1 for _ in range(SAMPLE_SIZE)] + [0 for _ in range(SAMPLE_SIZE)])

In [13]:
x_valid = np.concatenate([left_valid_set, right_valid_set])
labels_valid = np.asarray([1 for _ in range(VALID_SIZE)] + [0 for _ in range(VALID_SIZE)])

In [14]:
x_train.shape

(2000, 125, 94, 4)

In [15]:
labels_train.shape

(2000,)

Convolutional Model

In [16]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(48, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Conv2D(48, kernel_size=3, activation='relu')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

In [17]:
customAdam = keras.optimizers.Adam(lr=1e-6)
conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])

In [22]:
print('# Fit model on training data')

history = conv_model.fit(x_train, 
                    labels_train, 
                    batch_size=50, 
                    epochs=20,
                    validation_data=(x_valid, labels_valid),
                    shuffle = True)

# Fit model on training data


ValueError: Error when checking input: expected ani_image to have shape (125, 94, 3) but got array with shape (125, 94, 4)